In [1]:
from __future__ import print_function
import operator
import numpy as np
from graphviz import Digraph
from queue import Queue
import copy

In [2]:
import os
os.environ["PATH"] += os.pathsep + 'D:/Graphviz2.38/bin'

In [4]:
class Tree_expression:
    """
    A recursive implementation of Binary Tree
    Using links and Nodes approach.
    Modified to allow for trees to be constructed from other trees rather than always creating
    a new tree in the insertLeft or insertRight
    """
    
    def __init__(self,rootObj,label, index, parent=None):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None
        self.parent = parent
        self.label = label
        self.index = index
        
    def insertLeft(self, newNode, label, itera, parent=None):

        if isinstance(newNode, Tree_expression):
            t = newNode
        else:
            t = Tree_expression(newNode, label, itera, parent)

        if self.leftChild is not None:
            t.left = self.leftChild

        self.leftChild = t

    def insertRight(self,newNode, label, itera, parent=None):
        if isinstance(newNode,Tree_expression):
            t = newNode
        else:
            t = Tree_expression(newNode, label, itera, parent)

        if self.rightChild is not None:
            t.right = self.rightChild
        self.rightChild = t

    def isLeaf(self):
        return ((not self.leftChild) and (not self.rightChild))

    def getRightChild(self):
        return self.rightChild

    def getLeftChild(self):
        return self.leftChild
    
    def getLabel(self):
        return self.label
    
    def getIndex(self):
        return self.index

    def setRootVal(self,obj):
        self.key = obj

    def getRootVal(self):
        return self.key

    def inorder(self):
        if self.leftChild:
            self.leftChild.inorder()
        print(self.key)
        if self.rightChild:
            self.rightChild.inorder()

    def postorder(self):
        if self.leftChild:
            self.leftChild.postorder()
        if self.rightChild:
            self.rightChild.postorder()
        print(self.key)


    def preorder(self):
        print(self.key)
        if self.leftChild:
            self.leftChild.preorder()
        if self.rightChild:
            self.rightChild.preorder()                
                
    def printexp(self):
        #breakpoint()
        if self.leftChild:
            #if self.leftChild.getRootVal() in ('cos', 'sen', 'tan', 'log'):
            if self.getRootVal() in ('cos', 'sen', 'tan', 'log'):
               print(self.key, end=' ')
               print('(', end=' ')
               self.leftChild.printexp()
            else:
               print('(', end=' ')
               self.leftChild.printexp()
        if self.getRootVal() not in ('cos', 'sen', 'tan', 'log'):        
            print(self.key, end=' ')
        if self.rightChild:
            self.rightChild.printexp()
            print(')', end=' ')
    
    def postordereval(self):
        opers = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.truediv}
        res1 = None
        res2 = None
        if self.leftChild:
            res1 = self.leftChild.postordereval()  
        if self.rightChild:
            res2 = self.rightChild.postordereval() 
        if res1 and res2:
            return opers[self.key](res1,res2) 
        else:
            return self.key

In [5]:
class ParseTree(object):
    
    def __init__(self):
        self.graph = Digraph('G', filename='process1.gv',  node_attr={'shape': 'circle', 'height': '.1'})
        self.list_opera = []
        
    def buildParseTree(self, fpexp):
        fplist = fpexp.split()
        pStack = []
        itera = 0
        eTree = Tree_expression('','name' + str(itera), itera, None)
        itera +=1
        pStack.append(eTree)
        currentTree = eTree
        
        for idx,i in enumerate(fplist):
            if i == '(':
                
                currentTree.insertLeft('', 'name' + str(itera), itera, currentTree)
                itera +=1
                pStack.append(currentTree)
                currentTree = currentTree.getLeftChild()
            elif i in ['cos', 'sen', 'tan', 'log']:
                
                currentTree.setRootVal(i)
                
            elif i not in ['+', '-', '*', '/', ')']:
                if 'var_' in i:
                    currentTree.setRootVal(i)
                    parent = pStack.pop()
                    currentTree = parent
                else:
                    currentTree.setRootVal(int(i))
                    parent = pStack.pop()
                    currentTree = parent
            elif i in ['+', '-', '*', '/']:
                currentTree.setRootVal(i)
                currentTree.insertRight('','name' + str(itera), itera, currentTree)
                itera +=1
                pStack.append(currentTree)
                currentTree = currentTree.getRightChild()
            elif i == ')':
                currentTree = pStack.pop()
            else:
                raise ValueError
              
        if eTree.getRootVal() != '':
            return eTree
        else:
            eTree = eTree.leftChild
            eTree.parent = None
            eTree.index = 0
            eTree.label = 'name0'
            return eTree

    def height(self, tree):
         if tree == None:
            return -1
         else:
            return 1 + max(self.height(tree.leftChild),self.height(tree.rightChild))

    def evaluate(self, parseTree):
          opers = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.truediv, 
                   'cos': np.cos, 'sen': np.sin, 'tan': np.tan, 'log': np.log}
    
          leftC = parseTree.getLeftChild()
          rightC = parseTree.getRightChild()
          if leftC and rightC: 
              fn = opers[parseTree.getRootVal()]
              return fn(self.evaluate(leftC),self.evaluate(rightC))
             
          else:
              if parseTree.getRootVal() in ['cos', 'sen', 'tan', 'log']:
                 fn = opers[parseTree.getRootVal()]
                 return fn(self.evaluate(leftC))
              else: 
                 return parseTree.getRootVal()   
       
    def print_tree(self, parseTree):
        self.list_opera = []
        self._print_tree(parseTree)

    def _print_tree(self, parseTree):
        
        if parseTree.getLabel() not in self.list_opera:
                self.list_opera.append(parseTree.getLabel())
                
        if parseTree.leftChild and parseTree.rightChild:
            self.list_opera.append(parseTree.leftChild.getLabel())
            self.list_opera.append(parseTree.rightChild.getLabel())
            
            self.graph.node(parseTree.getLabel(), label=str(parseTree.getRootVal()))
            self.graph.node(parseTree.leftChild.getLabel(), label=str(parseTree.leftChild.getRootVal()))
            self.graph.node(parseTree.rightChild.getLabel(), label=str(parseTree.rightChild.getRootVal()))
            
            self.graph.edge(str(parseTree.getLabel()), str(parseTree.leftChild.getLabel()))
            self.graph.edge(str(parseTree.getLabel()), str(parseTree.rightChild.getLabel()))
            self._print_tree(parseTree.leftChild)
            self._print_tree(parseTree.rightChild)
            #print('quarto')
        elif parseTree.leftChild:
            
            self.list_opera.append(parseTree.leftChild.getLabel())
            self.graph.node(parseTree.getLabel(), label=str(parseTree.getRootVal()))
            self.graph.node(parseTree.leftChild.getLabel(), label=str(parseTree.leftChild.getRootVal()))
            self.graph.edge(str(parseTree.getLabel()), str(parseTree.leftChild.getLabel()))
            self._print_tree(parseTree.leftChild)
            
        else:
            pass
            
    def call_print_index(self, parseTree):
        
        self.lista_opera_index = list()
        self.print_index_list(parseTree)
        
    
    def print_index_list(self, parseTree):
    
        if parseTree.getIndex() not in self.lista_opera_index:
                self.lista_opera_index.append(parseTree.getIndex())
            
        if parseTree.leftChild and parseTree.rightChild:
                    try:
                        if parseTree.leftChild.getIndex() not in self.lista_opera_index:
                            self.lista_opera_index.append(parseTree.leftChild.getIndex())
                    except:
                        pass
                    try:
                        if parseTree.rightChild.getIndex() not in self.lista_opera_index:    
                            self.lista_opera_index.append(parseTree.rightChild.getIndex())
                    except:
                        pass
                    
                    self.print_index_list(parseTree.leftChild)
                    self.print_index_list(parseTree.rightChild)
        elif parseTree.leftChild:
                    try:
                        
                        if parseTree.rightChild.getIndex() not in self.lista_opera_index:   
                            self.lista_opera_index.append(parseTree.leftChild.getIndex())
                    except:
                        pass
                    self.print_index_list(parseTree.leftChild)
            
        else:
                    pass
                    
    
    def print_tree_index(self, parseTree):
        self.list_opera = []
        self._print_tree_index(parseTree)
        
        
    def _print_tree_index(self, parseTree):
        
        if parseTree.getLabel() not in self.list_opera:
                self.list_opera.append(parseTree.getLabel())
            
        
        if parseTree.leftChild and parseTree.rightChild:
        
            self.list_opera.append(parseTree.leftChild.getLabel())
            self.list_opera.append(parseTree.rightChild.getLabel())
            self.graph.node(parseTree.getLabel(), label=str(parseTree.getIndex()))
            self.graph.node(parseTree.leftChild.getLabel(), label=str(parseTree.leftChild.getIndex()))
            self.graph.node(parseTree.rightChild.getLabel(), label=str(parseTree.rightChild.getIndex()))
            
            self.graph.edge(str(parseTree.getLabel()), str(parseTree.leftChild.getLabel()))
            self.graph.edge(str(parseTree.getLabel()), str(parseTree.rightChild.getLabel()))
            self._print_tree_index(parseTree.leftChild)
            self._print_tree_index(parseTree.rightChild)
            
        elif parseTree.leftChild:
            
        
            self.list_opera.append(parseTree.leftChild.getLabel())
            self.graph.node(parseTree.getLabel(), label=str(parseTree.getIndex()))
            self.graph.node(parseTree.leftChild.getLabel(), label=str(parseTree.leftChild.getIndex()))
            self.graph.edge(str(parseTree.getLabel()), str(parseTree.leftChild.getLabel()))
            self._print_tree_index(parseTree.leftChild)
            
        else:
            pass

In [11]:
pt_2 = ParseTree()
parseTree_2 = pt_2.buildParseTree(" ( ( 2 - ( var_10 / var_11 ) ) + ( 7 * cos (  sen ( ( var_10 + var_11 ) ) ) ) ) ")

#parseTree_2 = pt_2.buildParseTree(" ( ( 7 * cos (  sen ( ( var_10 + var_11 ) ) ) ) ) ")
#pt_2.print_tree(parseTree_2)
pt_2.print_tree(parseTree_2)
pt_2.graph.view()
pt_2.graph.clear()
#parseTree_2.printexp()

In [12]:
parseTree_2.printexp()

( ( 2 - ( var_10 / var_11 ) ) + ( 7 * cos ( sen ( ( var_10 + var_11 ) ) ) 

In [13]:
def call_pars_index(parseTree,search_index):
    return search_index_parsetree(parseTree,search_index)


def search_index_parsetree(parseTree, search_index):
    q = Queue()  
    q.put(parseTree)  
    while (q.empty() == False): 

        node = q.queue[0]  
        if (node.index == search_index):
            node.parent = None
            return node

        q.get() 
        if (node.leftChild != None): 
            q.put(node.leftChild)  
        if (node.rightChild != None): 
            q.put(node.rightChild)

    return False 


def call_pars_index_mutation(parseTree_O, parseTree_M, search_index):

    q = Queue()  
    q.put(parseTree_O)
    pStack = list()

    indexes = list()
    while (q.empty() == False): 

        node = q.queue[0]
        if (node.index == search_index):
            node = parseTree_M
            pStack.append(node)
            return pStack
        q.get()
        if (node.leftChild != None):
            if node.leftChild.index == search_index:
                node.leftChild = parseTree_M
                pStack.append(node)
                return pStack

            q.put(node.leftChild)  
        if (node.rightChild != None): 
            if node.rightChild.index == search_index:
                node.rightChild = parseTree_M
                pStack.append(node)
                return pStack
            q.put(node.rightChild)
    return False

def rever_tree(dc, flag = 1):
    dc_t = True
    interm = dc[0].parent
    if flag == 1:
        if interm and (dc[0].leftChild or dc[0].rightChild):
            #print('ENTROU')
            while dc_t:
                if interm.parent:
                   interm = interm.parent
                else:
                   dc_t = False
            return interm    
        else:
            return dc[0]
    
    else:
        if interm:
            while dc_t:
                if interm.parent:
                   interm = interm.parent
                else:
                   dc_t = False
            return interm    
        else:
            return dc[0]
        
def refactory_index(parseTree, iteri):
    q = Queue()  
    q.put(parseTree)
    pstack = []
    parent = None
    while (q.empty() == False): 

        node = q.queue[0] 
        node.index = iteri
        node.label = 'name' + str(iteri)
        iteri += 1
        q.get() 
        if (node.leftChild != None):
            node.leftChild.parent = node
            q.put(node.leftChild)  
        if (node.rightChild != None): 
            q.put(node.rightChild)
            node.rightChild.parent = node
    pstack.append(node)
    return pstack 

def search_index(particles):
    roll_part = {}
    for key in particles.keys():
        pt, parsetree = particles[key] 
        pt.call_print_index(parsetree)
        roll_part[key] = np.random.choice(pt.lista_opera_index, 1)[0]
    return roll_part

def parses_trees(list_particles, lista):
    parses = {}
    for key in list_particles.keys():
        
       print(f'{key}') 
       _, parsetree = list_particles[key]
       parse = call_pars_index(parsetree,lista[key])
       parses[key] = parse
    return parses    

In [14]:
def choose_operations(lista, qtd = None, qtd_sub = 5, level = 0):
    prof = list()
    if level == 0: 
        if qtd == None:
            qtd = np.random.choice(np.arange(0,10),1) 
        
        cho = np.random.choice(lista, qtd)
     
        for ind in list(cho):
             n = np.random.choice(np.arange(0,qtd_sub),1)  
             prof.append((ind,np.random.choice(lista, n)))
    else:
        
        cho = np.random.choice(lista, 1)
        return cho
    return prof

def create_exp(exp):
    opers = {'sum':'+','sub':'-', 'mult':'*', 'div':'/', 
              'log': 'log', 'sen': 'sen', 'cos': 'cos',
             'tan': 'tan'}
    
    lista_app = list()
    for idx, expr in enumerate(exp):
        if expr in ['sum', 'sub', 'mult', 'div']:
            lista_app.append({1: '( ' + choose_features() + ' ' + opers[expr] + ' ' + choose_features() + ' )'})
            
        else:
            lista_app.append({2:opers[expr]})
    return lista_app


def choose_features():
    lista = ['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7']
    lista1 = np.arange(0,20)
    num = np.random.choice(lista1,1)
    qtd = np.random.choice(lista,1)
    return np.random.choice([num[0],qtd[0]],1)[0]
    
def buildParseTree_tree(lista):
    
    tam = len(lista)
    flag = 0
    expre = '('
    for idx in np.arange(0, tam - 1):
        
        if (list(lista[idx].keys())[0] == 1):
            if (list(lista[idx+1].keys())[0] == 1):
                expre = expre + ' ' + '(' + ' ' + lista[idx][1] + ' ' + '+'
                flag += 1
            elif (list(lista[idx+1].keys())[0] != 1):
                expre = expre + ' ' + lista[idx][1] + ' '
            
                for i in np.arange(0,flag):
                    flag -= 1
                    expre = expre + ')' + ' '
                expre = expre + '+'   \
                
        elif (list(lista[idx].keys())[0] == 2):
                 
            if (list(lista[idx+1].keys())[0] == 2):
                  flag += 1
                  expre = expre + ' ' + lista[idx][2] + ' ' + '(' 
                
            elif (list(lista[idx+1].keys())[0] != 2):
                  expre = expre + ' ' + lista[idx][2] + ' ' + '(' + ' ' + '(' + ' ' + choose_features() + ' ' + '-' + ' ' + choose_features() + ' ' \
                  + ')' + ' ' + ')' + ' '
                  for i in np.arange(0,flag):
                     flag -= 1
                     expre = expre + ')' + ' ' 
                  expre = expre + '+'
                        
    if (list(lista[tam-1].keys())[0] == 1):
        if (list(lista[tam-2].keys())[0] == 2):
            expre = expre + ' ' + lista[tam-1][1] + ' '
            for i in np.arange(0,flag):
                expre = expre + ')' + ' '
        else:
            expre = expre + ' ' + lista[tam-1][1] + ' '
            for i in np.arange(0,flag):
                expre = expre + ')' + ' '
    
    elif (list(lista[tam-1].keys())[0] == 2):
        if (list(lista[tam-2].keys())[0] == 2):
           expre = expre + ' ' + lista[tam-1][2] + ' ' + '(' + ' ' + '(' + ' ' + choose_features() + ' ' + '-' + ' ' + choose_features() + ' ' + ')' + ' ' + ')' + ' '
           for i in np.arange(0,flag):
                expre = expre + ')' + ' ' 
        elif (list(lista[tam-2].keys())[0] != 2):
            
            expre = expre + ' ' + lista[tam-1][2] + ' ' + '(' + ' ' + '(' + ' ' + choose_features() + ' ' + '-' + ' ' + choose_features() + ' ' + ')' + ' ' + ')' + ' '
            for i in np.arange(0,flag):
                expre = expre + ')' + ' ' 
    expre = expre + ')'
    return expre
        

In [26]:
lista = ['sum','sub','mult','div','log', 'tan', 'sen', 'cos']
cho = choose_operations(lista, qtd = 10, qtd_sub = 10)

In [27]:
list_dc = []
for ope in cho:
    lista = list()
    lista.append(ope[0])
    [lista.append(var) for var in ope[1]]
    dc = create_exp(lista)
    #print(dc)
    dc1 = list()
    dc2 = list()
    [dc1.append(var) if list(var.keys())[0] == 1 else dc2.append(var) for var in dc]
    dc = dc1 + dc2
    #print(dc)
    dc = buildParseTree_tree(dc)
    print(dc)
    pt_1 = ParseTree()
    parseTree_1 = pt_1.buildParseTree(dc)
    list_dc.append((copy.deepcopy(pt_1),copy.deepcopy(parseTree_1)))

( ( ( 7 + var_4 ) + ( ( 19 * 6 ) + ( ( 1 + var_1 ) + ( ( var_2 + 12 ) + ( var_3 - var_7 ) ) ) ) ) + log ( tan ( cos ( cos ( tan ( ( 12 - 19 ) ) ) ) ) ) )
( ( ( 6 * 10 ) + ( ( 13 / 13 ) + ( var_7 - var_4 ) ) ) + log ( tan ( log ( cos ( ( 4 - 4 ) ) ) ) ) )
( ( ( var_7 - 6 ) + ( var_5 - var_3 ) ) + cos ( ( var_6 - 15 ) ) )
( ( ( 15 * 14 ) + ( ( var_7 - 10 ) + ( ( var_7 * var_3 ) + ( ( 15 - 9 ) + ( var_4 / var_2 ) ) ) ) ) + sen ( sen ( log ( log ( ( var_1 - 0 ) ) ) ) ) )
( log ( tan ( log ( log ( ( 18 - var_3 ) ) ) ) ) )
( ( var_6 / var_5 ) + cos ( ( var_5 - var_6 ) ) )
( ( 4 + var_4 ) + sen ( ( var_6 - 17 ) ) )
( ( ( var_4 + var_2 ) + ( ( var_4 - var_6 ) + ( ( 1 + var_7 ) + ( ( 18 * 2 ) + ( 4 - var_5 ) ) ) ) ) + sen ( log ( ( var_1 - 19 ) ) ) )
( ( ( var_2 * var_5 ) + ( ( var_5 * 6 ) + ( var_1 - 11 ) ) ) + cos ( tan ( cos ( ( var_6 - 16 ) ) ) ) )
( ( ( 18 + var_4 ) + ( ( 12 / var_6 ) + ( 2 + var_1 ) ) ) + tan ( cos ( tan ( ( 10 - 4 ) ) ) ) )


In [28]:
tree = 0
list_dc[tree][0].print_tree(list_dc[tree][1])
list_dc[tree][0].graph.view()
list_dc[tree][0].graph.clear()

In [51]:
tree = 4
list_dc[tree][1].printexp()

log ( tan ( log ( log ( ( 18 - var_3 ) 

### Codigo abaixo de teste

In [8]:
#pt_1 = ParseTree()
#parseTree_1 = pt_1.buildParseTree("( ( 10 + 5 ) * 3 )")
#pt_1.print_tree_1(parseTree_1)
#pt_1.graph.view()
#pt_1.graph.clear()
##########################################
pt_2 = ParseTree()
parseTree_2 = pt_2.buildParseTree(" ( ( 2 - ( var_10 / var_11 ) ) + ( 7 * cos (  sen ( ( var_10 + var_11 ) ) ) ) ) ")
#pt_2.print_tree(parseTree_2)
pt_2.print_tree(parseTree_2)
pt_2.graph.view()
pt_2.graph.clear()
parseTree_2.printexp()
#######################################
#pt_3 = ParseTree()
#parseTree_3 = pt_3.buildParseTree(" ( ( ( 2 - ( 10 / 5 ) ) - sen ( ( 20 - 10 ) ) ) + cos ( cos \
#                              ( sen ( 10 ) ) ) )")
#pt_3.print_tree_1(parseTree_3)
#pt_3.graph.view()
#pt_3.graph.clear()

In [9]:
parseTree_2.printexp()

( ( 2 - ( var_10 / var_11 ) ) + ( 7 * cos ( ( ( var_10 + var_11 ) sen cos ) ) 

In [1062]:
list_particles  = {'par_1':[copy.deepcopy(pt_1),copy.deepcopy(parseTree_1)],
                   'par_2':[copy.deepcopy(pt_2),copy.deepcopy(parseTree_2)],
                   'par_3':[copy.deepcopy(pt_3),copy.deepcopy(parseTree_3)]}


lista2 = search_index(list_particles)
print(lista2)
parses = parses_trees(list_particles, lista2)
## faz a mutacao e retorna o root
dc = call_pars_index_mutation(copy.deepcopy(parseTree_2), parses['par_1'], lista2['par_2'])
dc = rever_tree(dc)
## refatora toda a arvore
dc2 = refactory_index(copy.deepcopy(dc), 0)
dc2 = rever_tree(dc2,0)
pt_1.print_tree_index(dc2)
pt_1.graph.view()
pt_1.graph.clear()

{'par_1': 0, 'par_2': 4, 'par_3': 3}
par_1
par_2
par_3


In [1060]:
pt_1.print_tree_index(dc2)
pt_1.graph.view()
pt_1.graph.clear()

In [56]:
pt_2.print_tree_index(parseTree_2)
pt_2.graph.view()
pt_2.graph.clear()

In [359]:
pt_3.print_tree(parseTree_3)
pt_3.graph.view()
pt_3.graph.clear()

In [ ]:
#pt_2.print_tree_1(parseTree_2)
#pt_2.print_tree(parseTree_2)
pt_2.graph.view()
pt_2.graph.clear()

In [ ]:
pt_1 = ParseTree()
parseTree_1 = pt_1.buildParseTree("( ( 10 + 5 ) * 3 )")
#parseTree_1.postorder()
#pt_1.print_tree_1(parseTree_1)
#pt_1.graph.view()
#pt_1.graph.clear()

pt_2 = ParseTree()
parseTree_2 = pt_2.buildParseTree(" ( ( 2 - ( 10 / 5 ) ) + ( 7 * cos (  sen ( ( 10 + 20 ) ) ) ) ) ")
#pt_2.print_tree_1(parseTree)
#pt_2.graph.view()
#pt_2.graph.clear()


pt_3 = ParseTree()
parseTree_3 = pt_3.buildParseTree(" ( ( ( 2 - ( 10 / 5 ) ) - sen ( ( 20 - 10 ) ) ) + cos ( cos \
                              ( sen ( 10 ) ) ) )")
#pt_3.print_tree_1(parseTree_3)
#pt_3.graph.view()
#pt_3.graph.clear()

list_particles  = {'par_1':[pt_1,parseTree_1],
                   'par_2':[pt_2,parseTree_2],
                   'par_3':[pt_3,parseTree_3]}


search_index(list_particles)
pt, parsetree = list_particles['par_1'] 

In [1064]:
pt_1 = ParseTree()
parseTree_1 = pt_1.buildParseTree("( ( 10 + 5 ) * 3 )")
#parseTree_1.postorder()
pt_1.print_tree(parseTree_1)
pt_1.graph.view()
pt_1.graph.clear()

In [1672]:
#pt_1 = ParseTree()
#parseTree_1 = pt_1.buildParseTree(list_dc[2])
tree = 2
list_dc[tree][0].print_tree(list_dc[tree][1])
list_dc[tree][0].graph.view()
list_dc[tree][0].graph.clear()

In [1449]:
lista2 = search_index(list_particles)

In [1450]:
lista2['par_1'], lista2['par_3']

(21, 8)

In [1452]:
list_particles  = {'par_1':[copy.deepcopy(list_dc[0][0]),copy.deepcopy(list_dc[0][1])],
                   'par_2':[copy.deepcopy(list_dc[1][0]),copy.deepcopy(list_dc[1][1])],
                   'par_3':[copy.deepcopy(list_dc[2][0]),copy.deepcopy(list_dc[2][1])]}


#lista2 = search_index(list_particles)
#print(lista2)
parses = parses_trees(list_particles, lista2)
## faz a mutacao e retorna o root
#dc = call_pars_index_mutation(copy.deepcopy(parseTree_2), parses['par_1'], lista2['par_2'])
dc = call_pars_index_mutation(copy.deepcopy(list_dc[0][1]), parses['par_3'], lista2['par_1'])
dc = rever_tree(dc)
## refatora toda a arvore
dc2 = refactory_index(copy.deepcopy(dc), 0)
dc2 = rever_tree(dc2,0)
list_dc[0][0].print_tree(dc2)
list_dc[0][0].graph.view()
list_dc[0][0].graph.clear()

par_1
par_2
par_3


In [1453]:
list_dc[0][0].evaluate(dc2)

214.73705182854752

In [1448]:
np.log(-10)

C:\Users\reblivi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


nan

In [1268]:
'''dc = list()
dc.append({2: 'log'})
dc.append({2: 'log'})
dc.append({1: '( 10 * 20 )'})
dc.append({1: '( 10 / 20 )'})
dc.append({1: '( 20 - 30 )'})'''
#dc.append({'all_1': 'sen'})
#dc.append({'all_1': 'tan'})
#dc.append({'all_1': 'sen'})
#dc.append({'all_1': 'log'})
#dc.append({'all': '( 90 - 50 )'})
#dc1 = buildParseTree_tree(dc)

In [1321]:
pt_1 = ParseTree()
#parseTree_1 = pt_1.buildParseTree("( sen ( tan ( ( 10 - 20 ) ) ) )")
parseTree_1 = pt_1.buildParseTree(dc1)
pt_1.print_tree(parseTree_1)
pt_1.graph.view()
pt_1.graph.clear()

In [628]:
pt_1.evaluate(parseTree_1)

-0.6038806745841075

In [694]:
pt_1 = ParseTree()
parseTree_1 = pt_1.buildParseTree("( sen ( ( 10 - 20 ) ) - sen ( ( 10 - 20 ) ) ) ")
pt_1.print_tree(parseTree_1)
pt_1.graph.view()
pt_1.graph.clear()

In [245]:
#parseTree_1 = pt_1.buildParseTree("( ( 10 + 5 ) * 3 )")

#parseTree_3 = pt_3.buildParseTree(" ( ( ( 2 - ( 10 / 5 ) ) - sen ( ( 20 - 10 ) ) ) + cos ( cos \
#                              ( sen ( 10 ) ) ) )")